# CSE 439 Assignment-1

# Student Information

Simge Pınar Erdoğan
20180702092

# Importing Frameworks and Libraries

In [1]:
import json
from base64 import b64encode
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes  
from base64 import b64decode
import sys
from flask import Flask
from flask import request
from flask_cors import CORS, cross_origin
from flask import *  
import time
import os
import random
import glob






# Defining Server Parameters

In [2]:
app = Flask(__name__)
app.secret_key = "abc"  
CORS(app)

# Defining Constent Parameters and nonceList

In [3]:
key =  str.encode("Thats my Kung Fu")
host = '127.0.0.1'
port = 161
nonce = 'Dx3SiuVkVJI='
nonceList = {}

# Defining functions

## Encryption of blocks based on given key and block number

In [4]:
def encryptBlock(): 
    
  #path of original text with given blocknumber
  blocksFolderName = './blocks'
    
  for blockFile in os.listdir(blocksFolderName):
      #reading the original text 
      blocksPath = blocksFolderName + '/'+blockFile   
      with open(blocksPath,encoding="utf8") as currentBlock:
          blockData = currentBlock.read()    
          blockData = str.encode(blockData)
          cipher = AES.new(key, AES.MODE_CTR)
          #encrpyting
          ct_bytes = cipher.encrypt(blockData)
          #taking the nonce
          nonce = b64encode(cipher.nonce).decode('utf-8')
          #inserting nonce to nonceList with given key in order to obtain in decrpytion part
          nonceList[blockFile]= nonce

          encrpytedBlock = b64encode(ct_bytes).decode('utf-8')

          #creating encrypted file
          encryptionFolderName = './encryptedBlocks/'
          encryptionPath = encryptionFolderName + blockFile

          file = open(encryptionPath, "w+")
          #writing encrpytedBlock 
          file.write(encrpytedBlock)

          file.close()

  print("Encryption is done succesfully")


# Creating blocks from book

In [5]:
def createBlocks():
    #desired book file
    fileName = 'book.txt'


    #Reading the book file and obtaining liens
    with open(fileName,encoding="utf8") as bookFile:
          linesList = bookFile.read().splitlines() 

    #filtering lines 
    linesList = list(filter(('').__ne__, linesList))
    byteArray = []

    #converting lines to bytes and inserting them to bytearray
    for currentLine in linesList:
        tempByteArray = bytearray(currentLine, 'utf-8')
        byteArray.append(tempByteArray)

    #Dividing byte array to desired length(16 byte) byte blocks 128 bits = 16 byte 
    byteBlocks = [byteArray[i:i+16] for i in range(0, len(byteArray), 16)]

    #creating byte blocks txt files of the book from byteBlocks array. 
    counter = 0 
    for currentBlock in byteBlocks:
        folderName = 'blocks'
        fileName = 'block-' + str(counter)
        path = './'+ folderName + '/'+ fileName+'.txt'
        #print("path => " + path)
        file = open(path, "wb+")

        for currentBytes in currentBlock:
          file.write(currentBytes)

        file.close()
        counter = counter + 1



# Main

In [6]:
def getSessionId():     
    if 'id' in session:
        return session['id']
    else:        
        #current time in miliseconds in order to create unique user id
        currentTimeMs = str(round(time.time() * 1000))
        session['id'] = currentTimeMs
        return currentTimeMs

In [7]:
def transferFiles(notSendBlockNumber):
  blocksArray = []  
  #path of original text with given blocknumber
  blocksFolderName = './encryptedBlocks'
  notSendFile = "block-"+str(notSendBlockNumber)+".txt"

  for blockFile in os.listdir(blocksFolderName):
      #reading the encrypted text 
      blocksPath = blocksFolderName + '/'+blockFile   
      if blockFile != notSendFile:  
          with open(blocksPath,encoding="utf8") as currentBlock:
              blockData = currentBlock.read()  
              currentData = {
                  "data":blockData,
                  "fileName":blockFile
              }   
              blocksArray.append(currentData)
                
  returnData = {
      "data":blocksArray
  }               
  return returnData
    

In [8]:
def getBlockCount():
    path, dirs, files = next(os.walk("./blocks"))
    return len(files)

In [9]:
@app.route('/')
def getFiles():
    totalBlockCount = getBlockCount()
    notSendBlockNumber = random.randint(0, totalBlockCount)
    print(notSendBlockNumber)
    sessionId = getSessionId()
    
    returnData = transferFiles(notSendBlockNumber)
    returnData['key']=key.decode("utf-8") 
    returnData['totalBlockCount'] = totalBlockCount
    returnData['nonceList'] = nonceList
    
    return returnData
    
    
    


In [10]:
@app.route('/getMissing/<missingBlockNumber>')
def getMissing(missingBlockNumber):
   print(missingBlockNumber) 
   blocksFolderName = './encryptedBlocks'
   blockFile = "block-"+str(missingBlockNumber)+".txt"
   blocksPath = blocksFolderName + '/'+blockFile 

   with open(blocksPath,encoding="utf8") as currentBlock:
              blockData = currentBlock.read()  
              currentData = {
                  "data":blockData,
                  "fileName":blockFile
              }   
              return currentData

    

In [ ]:
if __name__ == '__main__':
    #load_model()  # load model at the beginning once only
    createBlocks()
    encryptBlock()
    app.run(host=host, port=port)

Encryption is done succesfully
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:161/ (Press CTRL+C to quit)


3134


127.0.0.1 - - [05/Jan/2021 20:16:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 20:16:56] "GET /getMissing/3134 HTTP/1.1" 200 -


3134
1893


127.0.0.1 - - [05/Jan/2021 20:17:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 20:17:40] "GET /getMissing/1893 HTTP/1.1" 200 -


1893
